In [7]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [8]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [9]:
%%capture pwd
!pwd

In [10]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-05-18-what-is-the-likely-impact-of-advertising-restrictions-on-obesity',
 300,
 500)

# Fig 1

In [43]:
df = (
    pd.read_csv("raw/ADS_bigcatexp.csv")
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
# df.columns = ["response", "percent"]
# df["percent"] *= 100
df = df.set_index("Unnamed: 0").stack().reset_index()
df.columns = ["parent", "id", "value"]

In [44]:
c = {
    "High sugar and alcohol": colors["eco-dot"],
    "Processed": colors["eco-background"],
    "Fast food": colors["eco-yellow"],
    "Home cooking": colors["eco-green"],
    "Supermarkets": colors["eco-turquiose"],
}

In [45]:
df = df.join(pd.DataFrame(c, index=["color"]).T, on="parent")

In [46]:
f = "fig1_food groups"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".json", "w").write(
    json.dumps(list(df.T.to_dict().values()))
)
if LOCAL:
    f1 = df
df.head()

,parent,id,value,color
0,High sugar and alcohol,Confectionery,153.1770,#F4134D
1,High sugar and alcohol,Soft Drinks,64.5109,#F4134D
2,High sugar and alcohol,Alcohol,54.2943,#F4134D
3,High sugar and alcohol,Desserts,32.0924,#F4134D
4,High sugar and alcohol,Biscuits,8.8821,#F4134D


`pyecharts`

In [51]:
import pyecharts.options as opts
from pyecharts.charts import Line

x_data = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
y_data = [820, 932, 901, 934, 1290, 1330, 1320]

In [60]:
(
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="",
        y_axis=y_data,
        symbol="emptyCircle",
        is_symbol_show=True,
        label_opts=opts.LabelOpts(is_show=False),
        areastyle_opts=opts.AreaStyleOpts(opacity=1, color="#C67570"),
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=False),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
    )
    .render("basic_area_chart.html")
)

'/home/jovyan/github/eco/ecovisualisations/articles/2021-05-18-what-is-the-likely-impact-of-advertising-restrictions-on-obesity/basic_area_chart.html'